# VII - Parallel and Distributed Execution
In this notebook, we will execute training across multiple nodes (or in parallel across a single node over multiple GPUs). We will train an image classification model with Resnet20 on the CIFAR-10 data set across multiple nodes in this notebook.

Azure Batch and Batch Shipyard have the ability to perform "gang scheduling" or scheduling multiple nodes for a single task. This is most commonly used for Message Passing Interface (MPI) jobs.

* [Setup](#section1)
* [Configure and Submit MPI Job and Submit](#section2)
* [Delete Multi-Instance Job](#section3)

<a id='section1'></a>

## Setup

Create a simple alias for Batch Shipyard

In [24]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working

In [25]:
shipyard

Usage: shipyard.py [OPTIONS] COMMAND [ARGS]...

  Batch Shipyard: Provision and Execute Docker Workloads on Azure Batch

Options:
  --version   Show the version and exit.
  -h, --help  Show this message and exit.

Commands:
  cert      Certificate actions
  data      Data actions
  fs        Filesystem in Azure actions
  jobs      Jobs actions
  keyvault  KeyVault actions
  misc      Miscellaneous actions
  pool      Pool actions
  storage   Storage actions


Read in the account information we saved earlier

In [26]:
import json
import os

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)
    
def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

account_info = read_json('account_information.json')

storage_account_key = account_info['storage_account_key']
storage_account_name = account_info['storage_account_name']
STORAGE_ALIAS = account_info['STORAGE_ALIAS']

Create the `resource_files` to randomly download train and test data for CNTK

In [27]:
import random
IMAGE_NAME = 'alfpark/cntk:2.0.rc2-gpu-1bit-sgd-python3.5-cuda8.0-cudnn5.1'

Create data set conversion scripts to be uploaded. On real production runs, we would already have this data pre-converted instead of converting at the time of node startup.

Additionally, we'll create an MPI helper script for executing the MPI job. This helper script does the following:
1. Ensures that there are GPUs available to execute the task.
2. Parses the `$AZ_BATCH_HOST_LIST` for all of the hosts participating in the MPI job and creates a `hostfile` from it
3. Computes the total number of slots (processors)
4. Sets the proper CNTK training directory, script and options
5. Executes the MPI job via `mpirun`

In [28]:
%%writefile run_cifar10.sh
#!/usr/bin/env bash

set -e
set -o pipefail

# get number of GPUs on machine
ngpus=$(nvidia-smi -L | wc -l)
echo "num gpus: $ngpus"

if [ $ngpus -eq 0 ]; then
    echo "No GPUs detected."
    exit 1
fi

# get number of nodes
IFS=',' read -ra HOSTS <<< "$AZ_BATCH_HOST_LIST"
nodes=${#HOSTS[@]}

# create hostfile
touch hostfile
>| hostfile
for node in "${HOSTS[@]}"
do
    echo $node slots=$ngpus max-slots=$ngpus >> hostfile
done

# compute number of processors
np=$(($nodes * $ngpus))

# print configuration
echo "num nodes: $nodes"
echo "hosts: ${HOSTS[@]}"
echo "num mpi processes: $np"

# set cntk related vars
trainscript=$AZ_BATCH_TASK_SHARED_DIR/TrainResNet_CIFAR10.py

# set training options
trainopts="--datadir $AZ_BATCH_NODE_SHARED_DIR/data --modeldir $AZ_BATCH_TASK_WORKING_DIR/output --network resnet20 --distributed True -q 1 -a 0"

# execute mpi job
/root/openmpi/bin/mpirun --allow-run-as-root --mca btl_tcp_if_exclude docker0 \
    -np $np --hostfile hostfile -x LD_LIBRARY_PATH\
    /bin/bash -c "source /cntk/activate-cntk; python -u $trainscript $trainopts $*"

Writing run_cifar10.sh


Move the files into a directory to be uploaded.

In [29]:
INPUT_CONTAINER = 'input-dist'
UPLOAD_DIR = 'dist_upload'

!rm -rf $UPLOAD_DIR
!mkdir -p $UPLOAD_DIR
!mv run_cifar10.sh $UPLOAD_DIR
!ls -alF $UPLOAD_DIR

total 12
drwxr-xr-x  2 nbuser nbuser 4096 Aug 11 12:19 ./
drwx------ 20 nbuser nbuser 4096 Aug 11 12:19 ../
-rw-r--r--  1 nbuser nbuser 1080 Aug 11 12:19 run_cifar10.sh


We will create the config structure to directly reference these files to ingress into Azure Storage. This obviates the need to call `blobxfer` as it will be done for us during pool creation.

In [30]:
config = {
    "batch_shipyard": {
        "storage_account_settings": STORAGE_ALIAS
    },
    "global_resources": {
        "docker_images": [
            IMAGE_NAME
        ],
        "files": [
            {
                "source": {
                    "path": UPLOAD_DIR
                },
                "destination": {
                    "storage_account_settings": STORAGE_ALIAS,
                    "data_transfer": {
                        "container": INPUT_CONTAINER
                    }
                }
            }
        ]
    }
}

Now we'll create the pool specification with a few modifications for this particular execution:
- `inter_node_communication_enabled` will ensure nodes are allocated such that they can communicate with each other (e.g., send and receive network packets)
- `input_data` specifies the scripts we created above to be downloaded into `$AZ_BATCH_NODE_SHARED_DIR/cifar10_data`
- `transfer_files_on_pool_creation` will transfer the `files` specified in `global_resources` to be transferred during pool creation (i.e., `pool add`)
- `resource_files` are the CNTK train and test data files
- `additional_node_prep_commands` are commands to execute for node preparation of all compute nodes. Our additional node prep command is to execute the conversion script we created in an earlier step above

**Note:** Most often it is better to scale up the execution first, prior to scale out. Due to the default Batch core quota of just 20 cores, we are using 3 `STANDARD_NC6` nodes. In real production runs, we'd most likely scale up to multiple GPUs within a single node (parallel execution) such as `STANDARD_NC12` or `STANDARD_NC24` prior to scaling out to multiple NC nodes (parallel and distributed execution).

In [31]:
POOL_ID = 'gpupool-multi-instance'

pool = {
    "pool_specification": {
        "id": POOL_ID,
        "vm_configuration": {
            "platform_image": {
                "publisher": "Canonical",
                "offer": "UbuntuServer",
                "sku": "16.04-LTS"
            },
        },
        "vm_size": "STANDARD_NC6",
        "vm_count": {
            "dedicated": 3
        },
        "ssh": {
            "username": "docker"
        },
        "inter_node_communication_enabled": True,
        "reboot_on_start_task_failed": False,
        "block_until_all_global_resources_loaded": True,
        "transfer_files_on_pool_creation": True,
        "input_data": {
            "azure_storage": [
                {
                    "storage_account_settings": STORAGE_ALIAS,
                    "container": INPUT_CONTAINER,
                    "destination": "$AZ_BATCH_NODE_SHARED_DIR"
                }
            ]
        },
        "resource_files": [
            {
                "file_path": "cifar_data_processing.py",
                "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/cifar_data_processing.py",
                "file_mode":'0777'
            },
            {
                "file_path": "convert_cifar10.sh",
                "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/convert_cifar10.sh",
                "file_mode":'0777'
            }
        ],
         "additional_node_prep_commands": [
            "/bin/bash convert_cifar10.sh {} $AZ_BATCH_NODE_SHARED_DIR/data".format(IMAGE_NAME),
             "chmod 777 $AZ_BATCH_NODE_SHARED_DIR/run_cifar10.sh"
        ]
    }
}

In [32]:
!mkdir config
write_json_to_file(config, os.path.join('config', 'config.json'))
write_json_to_file(pool, os.path.join('config', 'pool.json'))
print(json.dumps(config, indent=4, sort_keys=True))
print(json.dumps(pool, indent=4, sort_keys=True))

mkdir: cannot create directory 'config': File exists
{
    "batch_shipyard": {
        "storage_account_settings": "mystorageaccount"
    }, 
    "global_resources": {
        "docker_images": [
            "alfpark/cntk:2.0.rc2-gpu-1bit-sgd-python3.5-cuda8.0-cudnn5.1"
        ], 
        "files": [
            {
                "destination": {
                    "data_transfer": {
                        "container": "input-dist"
                    }, 
                    "storage_account_settings": "mystorageaccount"
                }, 
                "source": {
                    "path": "dist_upload"
                }
            }
        ]
    }
}
{
    "pool_specification": {
        "additional_node_prep_commands": [
            "/bin/bash convert_cifar10.sh alfpark/cntk:2.0.rc2-gpu-1bit-sgd-python3.5-cuda8.0-cudnn5.1 $AZ_BATCH_NODE_SHARED_DIR/data", 
            "chmod 777 $AZ_BATCH_NODE_SHARED_DIR/run_cifar10.sh"
        ], 
        "block_until_all_global_resources_loa

Create the pool, please be patient while the compute nodes are allocated.

In [33]:
shipyard pool add -y

2017-08-11 12:20:07,933 INFO - creating container: shipyardremotefs
2017-08-11 12:20:08,092 INFO - creating table: shipyarddht
2017-08-11 12:20:08,275 INFO - creating table: shipyardtorrentinfo
2017-08-11 12:20:08,322 INFO - creating table: shipyardgr
2017-08-11 12:20:08,375 INFO - creating table: shipyardregistry
2017-08-11 12:20:08,421 INFO - creating container: shipyardrf-batch71d77646ba-gpupool-multi-instance
2017-08-11 12:20:08,465 INFO - creating table: shipyardimages
2017-08-11 12:20:08,510 INFO - creating container: shipyardtor-batch71d77646ba-gpupool-multi-instance
2017-08-11 12:20:08,552 INFO - creating container: shipyardgr-batch71d77646ba-gpupool-multi-instance
2017-08-11 12:20:08,594 DEBUG - clearing table (pk=batch71d77646ba$gpupool-multi-instance): shipyarddht
2017-08-11 12:20:08,741 DEBUG - clearing table (pk=batch71d77646ba$gpupool-multi-instance): shipyardtorrentinfo
2017-08-11 12:20:08,787 DEBUG - clearing table (pk=batch71d77646ba$gpupool-multi-instance): shipyardgr

Ensure that all compute nodes are `idle` and ready to accept tasks:

In [34]:
shipyard pool listnodes

2017-08-11 12:40:49,746 INFO - compute nodes for pool gpupool-multi-instance
* node id: tvm-1783593343_1-20170811t122122z
  * state: ComputeNodeState.idle
  * scheduling state: SchedulingState.enabled
  * no errors
  * start task:
    * exit code: 0
    * started: 2017-08-11 12:22:57.407723+00:00
    * completed: 2017-08-11 12:30:21.000244+00:00
    * duration: 0:07:23.592521
  * vm size: standard_nc6
  * dedicated: True
  * ip address: 10.0.0.6
  * running tasks: 0
  * total tasks run: 0
  * total tasks succeeded: 0
* node id: tvm-1783593343_2-20170811t122122z
  * state: ComputeNodeState.idle
  * scheduling state: SchedulingState.enabled
  * no errors
  * start task:
    * exit code: 0
    * started: 2017-08-11 12:22:57.549985+00:00
    * completed: 2017-08-11 12:30:24.393521+00:00
    * duration: 0:07:26.843536
  * vm size: standard_nc6
  * dedicated: True
  * ip address: 10.0.0.5
  * running tasks: 0
  * total tasks run: 0
  * total tasks succeeded: 0
* node id: tvm-1783593343_3-201

<a id='section2'></a>

## Configure MPI Job and Submit

MPI jobs in Batch require execution as a multi-instance task. Essentially this allows multiple compute nodes to be used for a single task.

A few things to note in this jobs configuration:
- The `COMMAND` executes the `run_cifar10.sh` script that was uploaded earlier as part of the node preparation task.
- `auto_complete` is being set to `True` which forces the job to move from `active` to `completed` state once all tasks complete. Note that once a job has moved to `completed` state, no new tasks can be added to it.
- `multi_instance` property is populated which enables multiple nodes, e.g., `num_instances` to participate in the execution of this task. The `coordination_command` is the command that is run on all nodes prior to the `command`. Here, we are simply executing the Docker image to run the SSH server for the MPI daemon (e.g., orted, hydra, etc.) to initialize all of the nodes prior to running the application command.

In [35]:
JOB_ID = 'cntk-mpi-job'

# reduce the nubmer of epochs to 20 for purposes of this notebook
COMMAND = '$AZ_BATCH_NODE_SHARED_DIR/run_cifar10.sh -e 20'
jobs = {
    "job_specifications": [
        {
            "id": JOB_ID,
            "auto_complete": True,
            "tasks": [
                {
                    "image": IMAGE_NAME,
                    "remove_container_after_exit": True,
                    "command": COMMAND,
                    "gpu": True,
                    "multi_instance": {
                        "num_instances": "pool_current_dedicated",
                        "coordination_command": "/usr/sbin/sshd -D -p 23",
                        "resource_files": [
                            {
                                "file_path": "resnet_models.py",
                                "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/resnet_models.py",
                                "file_mode":'0777'
                            },
                            {
                                "file_path": "TrainResNet_CIFAR10.py",
                                "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/TrainResNet_CIFAR10.py",
                                "file_mode":'0777'
                            }
                        ],
                    },
                }
            ],
        }
    ]
}

In [36]:
write_json_to_file(jobs, os.path.join('config', 'jobs.json'))
print(json.dumps(jobs, indent=4, sort_keys=True))

{
    "job_specifications": [
        {
            "auto_complete": true, 
            "id": "cntk-mpi-job", 
            "tasks": [
                {
                    "command": "$AZ_BATCH_NODE_SHARED_DIR/run_cifar10.sh -e 20", 
                    "gpu": true, 
                    "image": "alfpark/cntk:2.0.rc2-gpu-1bit-sgd-python3.5-cuda8.0-cudnn5.1", 
                    "multi_instance": {
                        "coordination_command": "/usr/sbin/sshd -D -p 23", 
                        "num_instances": "pool_current_dedicated", 
                        "resource_files": [
                            {
                                "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/resnet_models.py", 
                                "file_mode": "0777", 
                                "file_path": "resnet_models.py"
                            }, 
                            {
                                "blob_source": "https://batchshipyardexampl

Submit the job and tail `stdout.txt`:

In [37]:
shipyard jobs add --tail stdout.txt

2017-08-11 12:40:57,376 INFO - Adding job cntk-mpi-job to pool gpupool-multi-instance
2017-08-11 12:40:57,957 INFO - uploading file /tmp/tmpG9mmhO as u'shipyardtaskrf-cntk-mpi-job/task-00000.shipyard.envlist'
2017-08-11 12:40:57,999 DEBUG - submitting 1 tasks (0 -> 0) to job cntk-mpi-job
2017-08-11 12:40:58,261 INFO - submitted all 1 tasks to job cntk-mpi-job
2017-08-11 12:40:58,469 DEBUG - attempting to stream file stdout.txt from job=cntk-mpi-job task=task-00000
8c36f6001f233d02345634dd4547f23abf9324b0c081c6319d314b8c32cdb536
num gpus: 1
num nodes: 3
hosts: 10.0.0.4 10.0.0.5 10.0.0.6
num mpi processes: 3

************************************************************
CNTK is activated.

Please checkout tutorials and examples here:
  /cntk/Tutorials
  /cntk/Examples

To deactivate the environment run

  source /root/anaconda3/bin/deactivate

************************************************************

************************************************************
CNTK is activated.

Plea

Using the command below we can check the status of our jobs. Once all jobs have an exit code we can continue. You can also view the **heatmap** of this pool on [Azure Portal](https://portal.azure.com) to monitor the progress of this job on the compute nodes under your Batch account.

<a id='section3'></a>

## Delete Multi-instance Job

Deleting multi-instance jobs running as Docker containers requires a little more care. We will need to first ensure that the job has entered `completed` state. In the above `jobs` configuration, we set `auto_complete` to `True` enabling the Batch service to automatically complete the job when all tasks finish. This also allows automatic cleanup of the running Docker containers used for executing the MPI job.

Special logic is required to cleanup MPI jobs since the `coordination_command` that runs actually detaches an SSH server. The job auto completion logic Batch Shipyard injects ensures that these containers are killed.

In [38]:
shipyard jobs listtasks

2017-08-11 13:10:34,253 INFO - list of tasks for job cntk-mpi-job
* task id: task-00000
  * job id: cntk-mpi-job
  * state: TaskState.completed
  * max retries: 0
  * retention time: 10675199 days, 2:48:05.477581
  * execution details:
    * pool id: gpupool-multi-instance
    * node id: tvm-1783593343_3-20170811t122122z
    * started: 2017-08-11 12:40:59.639256+00:00
    * completed: 2017-08-11 12:47:56.294652+00:00
    * duration: 0:06:56.655396
    * exit code: 0


Once we are sure that the job is completed, then we issue the standard delete command:

In [39]:
shipyard jobs del -y --termtasks --wait

2017-08-11 13:10:40,927 DEBUG - Skipping termination of completed task task-00000 on job cntk-mpi-job
2017-08-11 13:10:41,160 INFO - deleting job: cntk-mpi-job
2017-08-11 13:10:41,433 DEBUG - waiting for job cntk-mpi-job to delete
2017-08-11 13:10:42,827 INFO - job cntk-mpi-job does not exist


In [40]:
shipyard pool del -y --wait

2017-08-11 13:10:44,523 INFO - Deleting pool: gpupool-multi-instance
2017-08-11 13:10:44,775 DEBUG - clearing table (pk=batch71d77646ba$gpupool-multi-instance): shipyarddht
2017-08-11 13:10:45,013 DEBUG - clearing table (pk=batch71d77646ba$gpupool-multi-instance): shipyardtorrentinfo
2017-08-11 13:10:45,051 DEBUG - clearing table (pk=batch71d77646ba$gpupool-multi-instance): shipyardgr
2017-08-11 13:10:45,145 DEBUG - clearing table (pk=batch71d77646ba$gpupool-multi-instance): shipyardregistry
2017-08-11 13:10:45,225 DEBUG - clearing table (pk=batch71d77646ba$gpupool-multi-instance): shipyardimages
2017-08-11 13:10:45,303 DEBUG - clearing table (pk=batch71d77646ba$gpupool-multi-instance): shipyardperf
2017-08-11 13:10:45,342 DEBUG - deleting queue: shipyardgr-batch71d77646ba-gpupool-multi-instance
2017-08-11 13:10:45,567 DEBUG - deleting container: shipyardrf-batch71d77646ba-gpupool-multi-instance
2017-08-11 13:10:45,758 DEBUG - deleting container: shipyardtor-batch71d77646ba-gpupool-mul